### stage_model_classifier

base: 14./stage_model_classifier_withneighbor-newfeatures

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
#classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
classifiers = ['currank','avgrank','dice','lr','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
#load data
_trim = 0
_predictlen = 2
_include_final = False
include_str = '1' if _include_final else '0'
suffix = f'indy500-2013-2019-end{include_str}-t{_trim}'
output_file = f'stage-indy500-2013-2019-end{include_str}-t{_trim}-l{_predictlen}.csv'
stagedata = pd.read_csv(output_file)


stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Data columns (total 35 columns):
Unnamed: 0                   1312 non-null int64
target                       1312 non-null int64
eventid                      1312 non-null int64
car_number                   1312 non-null int64
stageid                      1312 non-null int64
firststage                   1312 non-null int64
pit_in_caution               1312 non-null int64
start_position               1312 non-null int64
start_rank                   1312 non-null int64
start_rank_ratio             1312 non-null float64
top_pack                     1312 non-null int64
bottom_pack                  1312 non-null int64
average_rank                 1312 non-null float64
average_rank_all             1312 non-null float64
change_in_rank               1312 non-null int64
change_in_rank_all           1312 non-null float64
rate_of_change               1312 non-null int64
rate_of_change_all           1312 non-null float64
l

In [5]:
stagedata.head(5)

,Unnamed: 0,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,1,-5,0,1,1,1,0,7,15,0.454545,...,22.910400,2,31,66.08150,0,0,0,2,0,0
1,2,-2,0,1,2,1,1,7,3,0.090909,...,23.822858,2,27,62.06770,-2,-4,0,2,-1,0
2,3,-5,0,1,3,1,0,7,17,0.515152,...,21.857882,2,32,91.23935,-1,0,0,-2,0,2
3,4,-4,0,1,4,1,0,7,10,0.303030,...,19.394133,2,32,61.19415,0,0,0,-8,7,-4
4,5,-5,0,1,5,1,0,7,15,0.454545,...,17.737505,2,30,60.85410,0,0,0,0,2,0


In [6]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)


events = set(stagedata['eventid'])

years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
eventsname = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(eventsname)}


for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    
retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix)
df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2013
[*] predict with currank model
precision=[0.         0.20481928 0.        ], recall=[0. 1. 0.], f1=[0.   0.34 0.  ], accuracy=0.20
[*] predict with avgrank model
precision=[0.3625     0.25       0.62162162], recall=[0.51785714 0.08823529 0.60526316], f1=[0.42647059 0.13043478 0.61333333], accuracy=0.47
[*] predict with dice model
precision=[0.31481481 0.11111111 0.48684211], recall=[0.30357143 0.11764706 0.48684211], f1=[0.30909091 0.11428571 0.48684211], accuracy=0.35
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.59615385 0.8        0.70786517], recall=[0.55357143 0.58823529 0.82894737], f1=[0.57407407 0.6779661  0.76363636], accuracy=0.69
[*] predict with lsvc model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.63636364 0.7826087  0.70707071], recall=[0.5        0.52941176 0.92105263], f1=[0.56       0.63157895 0.8       ], accuracy=0.70
[*] predict with lsvcl2 model


...

precision=[0.6        0.41666667 0.68421053], recall=[0.19565217 0.35714286 0.94791667], f1=[0.29508197 0.38461538 0.79475983], accuracy=0.62
[*] predict with lsvcl2 model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.5        0.25949367 1.        ], recall=[0.04347826 0.97619048 0.22916667], f1=[0.08       0.41       0.37288136], accuracy=0.35
[*] predict with rf model
precision=[0.66666667 0.53658537 0.71311475], recall=[0.30434783 0.52380952 0.90625   ], f1=[0.41791045 0.53012048 0.79816514], accuracy=0.67
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


precision=[0.76190476 0.55       0.70731707], recall=[0.34782609 0.52380952 0.90625   ], f1=[0.47761194 0.53658537 0.79452055], accuracy=0.68
[*] predict with xgb model
precision=[0.64285714 0.46808511 0.69105691], recall=[0.19565217 0.52380952 0.88541667], f1=[0.3        0.49438202 0.77625571], accuracy=0.63
rec:['Indy500-2019', 1128, 184, '+:96,0:42,-:46', 0.22826086956521738, 0.4673913043478261, 0.30978260869565216, 0.6793478260869565, 0.625, 0.3532608695652174, 0.6684782608695652, 0.6793478260869565, 0.6304347826086957]


In [7]:
df_event

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2013,1146,166,"+:76,0:34,-:56",0.204819,0.469880,0.349398,0.686747,0.698795,0.662651,0.704819,0.668675,0.656627
0,Indy500-2014,1117,195,"+:108,0:43,-:44",0.220513,0.456410,0.384615,0.641026,0.687179,0.620513,0.666667,0.641026,0.589744
0,Indy500-2015,1145,167,"+:74,0:41,-:52",0.245509,0.425150,0.269461,0.550898,0.568862,0.544910,0.604790,0.562874,0.610778
0,Indy500-2016,1086,226,"+:84,0:44,-:98",0.194690,0.402655,0.345133,0.535398,0.588496,0.460177,0.570796,0.544248,0.610619
0,Indy500-2017,1097,215,"+:64,0:76,-:75",0.353488,0.386047,0.334884,0.381395,0.423256,0.497674,0.544186,0.381395,0.506977
0,Indy500-2018,1153,159,"+:92,0:44,-:23",0.276730,0.459119,0.396226,0.704403,0.735849,0.704403,0.679245,0.691824,0.672956
0,Indy500-2019,1128,184,"+:96,0:42,-:46",0.228261,0.467391,0.309783,0.679348,0.625000,0.353261,0.668478,0.679348,0.630435


In [8]:
stagedata[(stagedata['eventid']==5) & (stagedata['car_number']==12)]

,Unnamed: 0,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
1002,1172,6,5,12,1,1,0,3,4,0.121212,...,8.527139,2,32,66.10705,0,0,0,0,0,0
1003,1173,-2,5,12,2,1,1,3,6,0.181818,...,11.242669,2,18,61.85245,0,0,0,0,0,0
1004,1174,8,5,12,3,1,0,3,1,0.030303,...,23.296174,2,44,117.01525,0,0,0,0,0,0
1005,1175,5,5,12,4,1,0,3,2,0.060606,...,20.554273,2,35,59.24225,0,0,0,0,0,0


In [9]:
### fix train
#load data
_trim = 0
_include_final = False
include_str = '1' if _include_final else '0'
suffix = f'indy500-2013-2019-end{include_str}-t{_trim}'
output_file = f'stage-indy500-2013-2019-end{include_str}-t{_trim}-l{_predictlen}.csv'
stagedata = pd.read_csv(output_file)
stagedata.fillna(0, inplace=True)
#stagedata.info()

cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)


events = set(stagedata['eventid'])

years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
eventsname = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(eventsname)}

#first 
eventid = events_id['Indy500-2018']
ignore_eventid = events_id['Indy500-2019']
stdata_2018 = stagedata[stagedata['eventid']!=ignore_eventid]

print('Testset = %s'%eventsname[eventid])

train, test, train_x, train_y, test_x, test_y = split_by_eventid(stdata_2018, eventid)
test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
#print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
#      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

#record
rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

acc = [0 for x in range(len(classifiers))]
for idx, clf in enumerate(classifiers):
    acc[idx] = classifier_model(clf)

rec.extend(acc)
print('rec:%s'%rec)

#new df
df = pd.DataFrame([rec],columns=cols)
retdf = pd.concat([retdf, df])        


eventid = events_id['Indy500-2019']
ignore_eventid = events_id['Indy500-2018']
stdata_2019 = stagedata[stagedata['eventid']!=ignore_eventid]

print('Testset = %s'%eventsname[eventid])

train2, test2, train_x2, train_y2, test_x, test_y = split_by_eventid(stdata_2019, eventid)
test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))

#record
rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

acc = [0 for x in range(len(classifiers))]
for idx, clf in enumerate(classifiers):
    acc[idx] = classifier_model(clf)

rec.extend(acc)
print('rec:%s'%rec)

#new df
df = pd.DataFrame([rec],columns=cols)
retdf = pd.concat([retdf, df]) 
retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2018
[*] predict with currank model
precision=[0.         0.27672956 0.        ], recall=[0. 1. 0.], f1=[0.         0.43349754 0.        ], accuracy=0.28
[*] predict with avgrank model
precision=[0.22857143 0.1        0.56140351], recall=[0.34782609 0.02272727 0.69565217], f1=[0.27586207 0.03703704 0.62135922], accuracy=0.46
[*] predict with dice model
precision=[0.18965517 0.28125    0.60869565], recall=[0.47826087 0.20454545 0.45652174], f1=[0.27160494 0.23684211 0.52173913], accuracy=0.39
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.30612245 0.73333333 0.84210526], recall=[0.65217391 0.25       0.86956522], f1=[0.41666667 0.37288136 0.85561497], accuracy=0.67
[*] predict with lsvc model
precision=[0.40540541 0.73076923 0.875     ], recall=[0.65217391 0.43181818 0.91304348], f1=[0.5        0.54285714 0.89361702], accuracy=0.74
[*] predict with lsvcl2 model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.66666667 0.61111111 0.83333333], recall=[0.08695652 0.75       0.92391304], f1=[0.15384615 0.67346939 0.87628866], accuracy=0.75
[*] predict with rf model
precision=[0.34       0.66666667 0.84615385], recall=[0.73913043 0.27272727 0.83695652], f1=[0.46575342 0.38709677 0.84153005], accuracy=0.67
[*] predict with lrbias model


...

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[1.         1.         0.53038674], recall=[0.04347826 0.02380952 1.        ], f1=[0.08333333 0.04651163 0.69314079], accuracy=0.54
[*] predict with rf model
precision=[0.56756757 0.51282051 0.76851852], recall=[0.45652174 0.47619048 0.86458333], f1=[0.5060241  0.49382716 0.81372549], accuracy=0.67
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.7s finished


precision=[0.6122449  0.51851852 0.78703704], recall=[0.65217391 0.33333333 0.88541667], f1=[0.63157895 0.4057971  0.83333333], accuracy=0.70
[*] predict with xgb model
precision=[0.5        0.51428571 0.68376068], recall=[0.34782609 0.42857143 0.83333333], f1=[0.41025641 0.46753247 0.75117371], accuracy=0.62
rec:['Indy500-2019', 969, 184, '+:96,0:42,-:46', 0.22826086956521738, 0.4673913043478261, 0.30978260869565216, 0.6793478260869565, 0.657608695652174, 0.5380434782608695, 0.6739130434782609, 0.7010869565217391, 0.6195652173913043]


,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2018,969,159,"+:92,0:44,-:23",0.276730,0.459119,0.389937,0.666667,0.742138,0.754717,0.666667,0.685535,0.641509
0,Indy500-2019,969,184,"+:96,0:42,-:46",0.228261,0.467391,0.309783,0.679348,0.657609,0.538043,0.673913,0.701087,0.619565


In [10]:
retdf.to_csv(f'stint_classifier_result_t2013-2017_t{_trim}-l{_predictlen}.csv', float_format='%.3f')